In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, cv2 
from multiprocessing import Pool
import threading

# This notebook creates a new dataset by resizing each single image

Initialize the Kaggle API with my Auth token and the dataset metadata

In [ ]:
! mkdir -p /root/.kaggle/
! cp ../input/api-token/kaggle.json /root/.kaggle/kaggle.json
! mkdir -p /kaggle/tmp/hpa_512x512_dataset
! kaggle datasets init -p /kaggle/tmp/hpa_512x512_dataset

In [ ]:
%%bash
echo "{
  \"title\": \"HPA: 512x512 dataset\",
  \"id\": \"tchaye59/HPA512x512DATASET\",
  \"licenses\": [
    {
      \"name\": \"CC0-1.0\"
    }
  ]
}" > /kaggle/tmp/hpa_512x512_dataset/dataset-metadata.json

In [ ]:
DATA_DIR = "../input/hpa-single-cell-image-classification/"
TRAIN_DIR = os.path.join(DATA_DIR,"train/")
TEST_DIR = os.path.join(DATA_DIR,"test/")

DS_DIR = '/kaggle/tmp/hpa_512x512_dataset/'
train_df = pd.read_csv('../input/hpa-single-cell-image-classification/train.csv')
test_df = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')

In [ ]:
# Runs a process in a thread
class Worker(threading.Thread):
    
    def __init__(self, process,args,pbar=None):
        super().__init__()
        self.pbar = pbar
        self.process = process
        self.args = args

    def run(self):
        res = self.process(self.args)
        if self.pbar:
            self.pbar.update(1)

# Build dataset

In [ ]:
def rgb_worker_fn(args):
    id_,src_path,dest_path,rgb_dest_dir = args
    dim = (512, 512)
    
    red_image = cv2.imread(src_path+id_+"_red.png", cv2.IMREAD_UNCHANGED)
    red_image = cv2.resize(red_image, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite(dest_path+id_+"_red.png" , red_image)
    
    green_image = cv2.imread(src_path+id_+"_green.png", cv2.IMREAD_UNCHANGED)
    green_image = cv2.resize(green_image, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite(dest_path+id_+"_green.png" , green_image)
    
    blue_image = cv2.imread(src_path+id_+"_blue.png", cv2.IMREAD_UNCHANGED)
    blue_image = cv2.resize(blue_image, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite(dest_path+id_+"_blue.png" , blue_image)
    
    yellow_image = cv2.imread(src_path+id_+"_yellow.png", cv2.IMREAD_UNCHANGED)
    yellow_image = cv2.resize(yellow_image, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite(dest_path+id_+"_yellow.png" , yellow_image)
        
    image = np.array([red_image, green_image, blue_image,])
    image = np.transpose(image, (1,2,0))
    #image=image.astype(np.uint8) 
    cv2.imwrite(rgb_dest_dir+id_+".png" , image)

# Training images

In [ ]:
ids = train_df.ID.values
src_dir  =  TRAIN_DIR
dest_dir = DS_DIR+'train/'
rgb_dest_dir =  DS_DIR+'rgb_train/'
os.makedirs(dest_dir,exist_ok=True)
os.makedirs(rgb_dest_dir,exist_ok=True)

In [ ]:
size = len(ids)
pbar = tqdm(total=size)
n = 1000
for i in range(0,size,n):
    workers = []
    for id_ in ids[i:min(size,i+n)]:
        worker =  Worker(rgb_worker_fn,(id_,src_dir,dest_dir,rgb_dest_dir),pbar=pbar)
        worker.start()
        workers.append(worker)
    for worker in workers:
        worker.join()

# Testing images

In [ ]:
ids = test_df.ID.values
src_dir  =  TEST_DIR
dest_dir = DS_DIR+'test/'
rgb_dest_dir =  DS_DIR+'rgb_test/'
os.makedirs(dest_dir,exist_ok=True)
os.makedirs(rgb_dest_dir,exist_ok=True)

In [ ]:
size = len(ids)
pbar = tqdm(total=size)
n = 1000
for i in range(0,size,n):
    workers = []
    for id_ in ids[i:min(size,i+n)]:
        worker =  Worker(rgb_worker_fn,(id_,src_dir,dest_dir,rgb_dest_dir),pbar=pbar)
        worker.start()
        workers.append(worker)
    for worker in workers:
        worker.join()

In [ ]:
! kaggle datasets version -p /kaggle/tmp/hpa_512x512_dataset -m "add rgb images"  --dir-mode tar
#! kaggle datasets create -p /kaggle/tmp/hpa_512x512_dataset/ -u --dir-mode tar

In [ ]:
! rm -rf /root/.kaggle/kaggle.json

Dataset link : https://www.kaggle.com/tchaye59/hpa512x512dataset